# Utils

> Fill in a module description here


In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.utils import *  # noqa: F403
from torch.utils.data import DataLoader
import numpy as np
import yaml
import torch
from fedai.data import *
import numpy as np

In [ ]:
#| export
import importlib
def get_class(module_name, class_name):
    module = importlib.import_module(module_name)
    return getattr(module, class_name)

In [ ]:
#| export
def load_ds(cfg, get_datasets):
    print("Generating the datasets...")
    loss_ds, gener_ds = get_datasets(cfg)
    print("Datasets generated successfully.")
    return loss_ds, gener_ds

In [ ]:
#| export
def get_server(cfg, lst_data_dict, model, holdout_ds, **kwargs):
    Server = get_class('fedai.servers', f'Server_{cfg.name}')
    client_class = get_class('fedai.clients', f'Client_{cfg.name}')
    return Server(cfg, lst_data_dict, model, holdout_ds, client_class, **kwargs)

In [ ]:
#| export
def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [ ]:
#| export
def save_space(server, client) -> None:
    client.clear_model()
    del client.optimizer
    del client
    import gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    server.client_list.clear_cache()

Because of the difference that it takes to prepare the dataset for sinle device vs multi-device training, we make a method that handles this separately. `prepare_dl` prepares the dataloader needed for the trainer's type.

In [ ]:
#| export
def prepare_dl(trainer, ds, shuffle=True):
    return DataLoader(
        ds,
        batch_size= trainer.cfg.batch_size,
        shuffle= shuffle,
        collate_fn= trainer.client.collat_fn        
    )

## A Lazy initializer 

To save the memory, we don't need to instantiate all the client's objects at once. We can use `generators` as our tool to **lazily** instanitate them, meaning that they will only be instantiated and created in memory when we access them. This can be achieved by creating a class ad overriding the `__getitem__` method of our defined class. Inside the class, we use not a generator directly, but a cache object (dictionary) to retireve the clients.

In [ ]:
#| export
class LazyList:
    def __init__(self, server, client_cls):
        self.server = server
        self.client_cls = client_cls
        self.client_cache = {}  # Cache to store initialized clients

    def clear_cache(self):
        # Clear the cache to free memory if needed
        self.client_cache = {}


In [ ]:
#| export
@patch
def __getitem__(self: LazyList, idx):
    # Check if the client is already instantiated
    if idx not in self.client_cache:
        # Instantiate the client and store it in the cache
        self.client_cache[idx] = self.client_cls(
            data_dict= self.server.lst_data_dict[idx],
            model= None, #deepcopy(self.server.model),
            criterion= self.server.criterion,
            optimizer= None, #get_class('torch.optim', self.server.cfg.optimizer)(self.server.model.parameters(), lr= self.server.cfg.lr),
            idx= idx,
            gen_data_dict= self.server.lst_gen_data_dict[idx],
            tokenizer= self.server.tokenizer,
            collat_fn= self.server.collat_fn,
            cfg= self.server.cfg
        )
    return self.client_cache[idx]

Let us see how can this be used on a real example.

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()